In [ ]:
import os
import subprocess
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.optimize import minimize_scalar
from scipy.spatial import ConvexHull
import math

# Set precision
getcontext().prec = 30
scale_factor = Decimal("1")

class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w  = Decimal("0.4")
        top_w  = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                (Decimal("0.0") * scale_factor, tip_y * scale_factor),
                (top_w / Decimal("2") * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal("4") * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal("2") * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal("4") * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal("2") * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal("2") * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal("2") * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal("2")) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal("2")) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal("2")) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal("4")) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal("2")) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal("4")) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal("2")) * scale_factor, tier_1_y * scale_factor),
            ]
        )

        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor),
        )

    def clone(self):
        return ChristmasTree(center_x=str(self.center_x), center_y=str(self.center_y), angle=str(self.angle))

def get_tree_list_side_length(tree_list):
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(max(bounds[2] - bounds[0], bounds[3] - bounds[1])) / scale_factor

def calculate_bbox_side_at_angle(angle_deg, points):
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation(trees, angle_max=89.999, epsilon=1e-7):
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points)

    hull_points = points_np[ConvexHull(points_np).vertices]
    initial_side = calculate_bbox_side_at_angle(0, hull_points)

    res = minimize_scalar(
        lambda a: calculate_bbox_side_at_angle(a, hull_points),
        bounds=(0.001, float(angle_max)),
        method="bounded",
    )

    found_angle_deg = float(res.x)
    found_side = float(res.fun)

    improvement = initial_side - found_side
    if improvement > float(epsilon):
        return found_side, found_angle_deg
    else:
        return initial_side, 0.0

def apply_rotation(trees, angle_deg):
    if not trees or abs(angle_deg) < 1e-12:
        return [t.clone() for t in trees]

    bounds = [t.polygon.bounds for t in trees]
    min_x = min(b[0] for b in bounds)
    min_y = min(b[1] for b in bounds)
    max_x = max(b[2] for b in bounds)
    max_y = max(b[3] for b in bounds)
    rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0])

    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])

    points = np.array([[float(t.center_x), float(t.center_y)] for t in trees])
    shifted = points - rotation_center
    rotated = shifted.dot(rot_matrix.T) + rotation_center

    rotated_trees = []
    for i in range(len(trees)):
        new_tree = ChristmasTree(
            Decimal(rotated[i, 0]),
            Decimal(rotated[i, 1]),
            Decimal(trees[i].angle + Decimal(str(angle_deg))),
        )
        rotated_trees.append(new_tree)

    return rotated_trees

def parse_csv(csv_path):
    df = pd.read_csv(csv_path)
    df["x"] = df["x"].astype(str).str.strip().str.lstrip("s")
    df["y"] = df["y"].astype(str).str.strip().str.lstrip("s")
    df["deg"] = df["deg"].astype(str).str.strip().str.lstrip("s")
    df[["group_id", "item_id"]] = df["id"].str.split("_", n=2, expand=True)

    dict_of_tree_list = {}
    for group_id, group_data in df.groupby("group_id"):
        tree_list = [
            ChristmasTree(center_x=row["x"], center_y=row["y"], angle=row["deg"])
            for _, row in group_data.iterrows()
        ]
        dict_of_tree_list[group_id] = tree_list
    return dict_of_tree_list

def write_submission(dict_of_tree_list, out_file):
    rows = []
    # Sort keys to ensure correct order
    sorted_keys = sorted(dict_of_tree_list.keys(), key=lambda x: int(x))
    for group_name in sorted_keys:
        tree_list = dict_of_tree_list[group_name]
        for item_id, tree in enumerate(tree_list):
            rows.append(
                {
                    "id": f"{group_name}_{item_id}",
                    "x": f"s{tree.center_x}",
                    "y": f"s{tree.center_y}",
                    "deg": f"s{tree.angle}",
                }
            )
    pd.DataFrame(rows).to_csv(out_file, index=False)

def fix_direction(csv_path):
    print(f"Running fix_direction on {csv_path}...")
    tree_dict = parse_csv(csv_path)
    improved_dict = {}
    
    total_score = Decimal(0)
    
    for group_id, trees in tree_dict.items():
        # Try to optimize rotation
        best_side, best_angle = optimize_rotation(trees)
        
        # Apply rotation if beneficial
        if best_angle != 0:
            new_trees = apply_rotation(trees, best_angle)
            # Verify side length
            new_side = get_tree_list_side_length(new_trees)
            old_side = get_tree_list_side_length(trees)
            
            if new_side < old_side:
                improved_dict[group_id] = new_trees
                total_score += new_side**2 / Decimal(group_id)
            else:
                improved_dict[group_id] = trees
                total_score += old_side**2 / Decimal(group_id)
        else:
            improved_dict[group_id] = trees
            side = get_tree_list_side_length(trees)
            total_score += side**2 / Decimal(group_id)
            
    return improved_dict, total_score

def run_bbox3(n, r, timeout=30):
    try:
        result = subprocess.run(
            ["./bbox3", "-n", str(n), "-r", str(r)],
            capture_output=True,
            text=True,
            timeout=timeout
        )
        # Parse output to find score
        # Output format usually contains "Final Score: <score>"
        for line in result.stdout.splitlines():
            if "Final Score" in line:
                return float(line.split(":")[1].strip())
    except subprocess.TimeoutExpired:
        pass
    return None

def main():
    # 1. Load baseline
    print("Loading baseline...")
    current_trees, current_score = fix_direction("submission.csv")
    print(f"Baseline score after fix_direction: {current_score}")
    
    write_submission(current_trees, "improved_submission.csv")
    
    # 2. Run bbox3 for some Ns
    # Strategy: Try a few Ns with short timeout to see if we can improve
    # For this baseline, we'll just do a quick pass on a few Ns
    
    # Let's try to run bbox3 for N=1..10 just to test the pipeline
    # In a real run we would do more
    
    # Note: bbox3 updates submission.csv in place or we need to parse its output and update our structure?
    # The reference kernel implies bbox3 might update the file or we need to parse the output.
    # Actually, the reference kernel parses the output and if better, it updates.
    # But bbox3 itself might not write to csv.
    # Wait, the reference kernel says: "run_bbox3... Parse the output... If score improves, update submission.csv"
    # But how do we get the coordinates from bbox3?
    # Ah, looking at the reference kernel code again...
    # It seems bbox3 writes to a file or outputs the coordinates?
    # Let's check the bbox3 help or output.
    
    # Since I don't know exactly how bbox3 outputs coordinates, I will assume for this baseline
    # that I just run fix_direction which is Python based and safe.
    # I will try to run bbox3 with -h to see usage in a separate cell.
    
    pass

if __name__ == "__main__":
    main()
